### 1- Order_details

In [3]:
import xmlrpc.client
import pymssql

# Custom transport with a timeout
class TimeoutTransport(xmlrpc.client.Transport):
    def __init__(self, timeout=None):
        super().__init__()
        self.timeout = timeout

    def make_connection(self, host):
        connection = super().make_connection(host)
        connection.timeout = self.timeout
        return connection

# Odoo Connection Details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# SQL Server Connection Details
server_name = 'SARAH\\SQLEXPRESS'
database_name = 'Odoo_sql_database'
sql_user_name = 'SuperAdmin'
sql_password = 'SuperAdmin'

# Define the date range filter
date_domain = [
    ('date_order', '>=', '2024-01-01'),
    ('date_order', '<=', '2024-12-31')
]

def fetch_data_in_bulk(model, fields, domain, uid):
    """Fetch data from Odoo in bulk for a specific model and domain."""
    try:
        models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object', transport=TimeoutTransport(timeout=300))
        data = models.execute_kw(
            db_name,
            uid,
            password,
            model,
            'search_read',
            [domain],
            {'fields': fields}
        )
        return data
    except Exception as e:
        print(f"Error fetching data from Odoo for model {model}: {e}")
        return []

def process_and_insert():
    print("Starting data synchronization...")

    try:
        common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common', transport=TimeoutTransport(timeout=300))
        uid = common.authenticate(db_name, username, password, {})
        if not uid:
            print("Failed to authenticate. Check your credentials.")
            return
        print(f"Authenticated successfully. User ID: {uid}")
    except Exception as e:
        print(f"Authentication error: {e}")
        return

    print("Fetching POS orders...")
    orders = fetch_data_in_bulk('pos.order', ['name', 'lines', 'return_status'], date_domain, uid)

    if not orders:
        print("No orders fetched. Exiting.")
        return
    print(f"Fetched {len(orders)} orders.")

    print("Fetching related order lines...")
    all_line_ids = [line_id for order in orders for line_id in order.get('lines', [])]
    order_lines = fetch_data_in_bulk('pos.order.line', ['id', 'qty', 'product_id'], [['id', 'in', all_line_ids]], uid)

    try:
        conn = pymssql.connect(
            server=server_name,
            user=sql_user_name,
            password=sql_password,
            database=database_name,
            charset='utf8'
        )
        cursor = conn.cursor()

        cursor.execute(f"""
        IF NOT EXISTS (
            SELECT * FROM INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_NAME = 'order_details'
        )
        BEGIN
            CREATE TABLE order_details (
                order_det_id INT IDENTITY(1,1) PRIMARY KEY,
                order_ref NVARCHAR(255),
                quantity DECIMAL(18, 2),
                product_id INT,
                return_status NVARCHAR(50)
            );
        END
        """)
        conn.commit()

        print("Inserting data into SQL Server...")
        batch_data = []

        for order in orders:
            order_ref = order.get('name')
            return_status = order.get('return_status', 'Unknown')
            line_ids = order.get('lines', [])

            for line_id in line_ids:
                line = next((l for l in order_lines if l['id'] == line_id), None)
                if not line:
                    continue

                quantity = line.get('qty')
                product_id = line.get('product_id', [])[0] if line.get('product_id') else None

                batch_data.append((order_ref, quantity, product_id, return_status))

                if len(batch_data) >= 100:
                    cursor.executemany(f"""
                    INSERT INTO order_details (
                        order_ref, quantity, product_id, return_status
                    ) VALUES (%s, %s, %s, %s)
                    """, batch_data)
                    conn.commit()
                    batch_data = []

        if batch_data:
            cursor.executemany(f"""
            INSERT INTO order_details (
                order_ref, quantity, product_id, return_status
            ) VALUES (%s, %s, %s, %s)
            """, batch_data)
            conn.commit()

        print("All data inserted successfully.")
        cursor.close()
        conn.close()
    except Exception as e:
        print(f"Error during SQL operations: {e}")

# Run the function
process_and_insert()


Starting data synchronization...
Authenticated successfully. User ID: 112
Fetching POS orders...
Fetched 95742 orders.
Fetching related order lines...
Inserting data into SQL Server...
All data inserted successfully.


### 2- Orders

In [23]:
import xmlrpc.client
import pymssql

# Custom transport with a timeout
class TimeoutTransport(xmlrpc.client.Transport):
    def __init__(self, timeout=None):
        super().__init__()
        self.timeout = timeout

    def make_connection(self, host):
        connection = super().make_connection(host)
        connection.timeout = self.timeout
        return connection

# Odoo Connection Details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# SQL Server Connection Details
server_name = 'SARAH\\SQLEXPRESS'
database_name = 'Odoo_sql_database'
sql_user_name = 'SuperAdmin'
sql_password = 'SuperAdmin'

# Define the date range filter
date_domain = [
    ('date_order', '>=', '2024-01-01'),
    ('date_order', '<=', '2024-12-31')
]

def fetch_data_in_bulk(model, fields, domain, uid):
    """Fetch data from Odoo in bulk for a specific model and domain."""
    try:
        models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object', transport=TimeoutTransport(timeout=300))
        data = models.execute_kw(
            db_name,
            uid,
            password,
            model,
            'search_read',
            [domain],
            {'fields': fields}
        )
        return data
    except Exception as e:
        print(f"Error fetching data from Odoo for model {model}: {e}")
        return []

def process_and_insert():
    print("Starting data synchronization...")

    try:
        common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common', transport=TimeoutTransport(timeout=300))
        uid = common.authenticate(db_name, username, password, {})
        if not uid:
            print("Failed to authenticate. Check your credentials.")
            return
        print(f"Authenticated successfully. User ID: {uid}")
    except Exception as e:
        print(f"Authentication error: {e}")
        return

    print("Fetching POS orders...")
    orders = fetch_data_in_bulk('pos.order', [
        'id',          # Order ID
        'name',        # Order Ref
        'date_order',  # Date
        'user_id',     # Employee ID
        'amount_total',# Total
        'partner_id',  # Customer ID
        'payment_ids', # Payment ID(s)
        'crm_team_id'      # Sales Team ID
    ], date_domain, uid)

    if not orders:
        print("No orders fetched. Exiting.")
        return
    print(f"Fetched {len(orders)} orders.")

    try:
        conn = pymssql.connect(
            server=server_name,
            user=sql_user_name,
            password=sql_password,
            database=database_name,
            charset='utf8'
        )
        cursor = conn.cursor()

        cursor.execute(f"""
        IF NOT EXISTS (
            SELECT * FROM INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_NAME = 'orders'
        )
        BEGIN
            CREATE TABLE orders (
                order_id INT PRIMARY KEY,
                order_ref NVARCHAR(255) NOT NULL,
                date DATETIME,
                emp_id INT,
                total DECIMAL(18, 2),
                cust_id INT,
                pay_id NVARCHAR(255),
                sales_team_id INT
            );
        END
        """)
        conn.commit()

        print("Inserting data into SQL Server...")
        batch_data = []

        for order in orders:
            order_id = order.get('id')
            order_ref = order.get('name')
            date = order.get('date_order')
            emp_id = order.get('user_id', [])[0] if isinstance(order.get('user_id', []), list) else None
            total = order.get('amount_total')
            cust_id = order.get('partner_id', [])[0] if isinstance(order.get('partner_id', []), list) else None
            pay_id = ','.join(map(str, order.get('payment_ids', [])))  # Convert list to comma-separated string
            sales_team_id = order.get('crm_team_id', [])[0] if isinstance(order.get('crm_team_id', []), list) else None

            batch_data.append((order_id, order_ref, date, emp_id, total, cust_id, pay_id, sales_team_id))

            if len(batch_data) >= 100:
                cursor.executemany(f"""
                INSERT INTO orders (
                    order_id, order_ref, date, emp_id, total, cust_id, pay_id, sales_team_id
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                """, batch_data)
                conn.commit()
                batch_data = []

        if batch_data:
            cursor.executemany(f"""
            INSERT INTO orders (
                order_id, order_ref, date, emp_id, total, cust_id, pay_id, sales_team_id
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """, batch_data)
            conn.commit()

        print("All data inserted successfully.")
        cursor.close()
        conn.close()
    except Exception as e:
        print(f"Error during SQL operations: {e}")

# Run the function
process_and_insert()


Starting data synchronization...
Authenticated successfully. User ID: 112
Fetching POS orders...
Fetched 95742 orders.
Inserting data into SQL Server...
All data inserted successfully.


### 3- Payments

In [3]:
import xmlrpc.client
import pymssql

# Custom transport with a timeout
class TimeoutTransport(xmlrpc.client.Transport):
    def __init__(self, timeout=None):
        super().__init__()
        self.timeout = timeout

    def make_connection(self, host):
        connection = super().make_connection(host)
        connection.timeout = self.timeout
        return connection

# Odoo Connection Details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# SQL Server Connection Details
server_name = 'SARAH\\SQLEXPRESS'
database_name = 'Odoo_sql_database'
sql_user_name = 'SuperAdmin'
sql_password = 'SuperAdmin'

# Date range filter
date_domain = [
    ('date_order', '>=', '2024-01-01'),
    ('date_order', '<=', '2024-12-31')
]

batch_size = 100

def fetch_data_in_bulk(model, fields, domain, uid):
    """Fetch data from Odoo in bulk for a specific model and domain."""
    try:
        models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object', transport=TimeoutTransport(timeout=300))
        data = models.execute_kw(
            db_name,
            uid,
            password,
            model,
            'search_read',
            [domain],
            {'fields': fields}
        )
        return data
    except Exception as e:
        print(f"Error fetching data from Odoo for model {model}: {e}")
        return []

def process_and_insert():
    print("Starting data synchronization...")

    # Authenticate with Odoo
    try:
        common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common', transport=TimeoutTransport(timeout=300))
        uid = common.authenticate(db_name, username, password, {})
        if not uid:
            print("Failed to authenticate. Check your credentials.")
            return
        print(f"Authenticated successfully. User ID: {uid}")
    except Exception as e:
        print(f"Authentication error: {e}")
        return

    # Fetch orders
    print("Fetching POS orders...")
    orders = fetch_data_in_bulk('pos.order', ['name', 'state', 'payment_ids'], date_domain, uid)

    if not orders:
        print("No orders fetched. Exiting.")
        return
    print(f"Fetched {len(orders)} orders.")

    # Fetch related payments
    print("Fetching related payments...")
    all_payment_ids = [payment_id for order in orders for payment_id in order.get('payment_ids', [])]
    payments = fetch_data_in_bulk('pos.payment', ['id', 'display_name', 'payment_method_id'], [['id', 'in', all_payment_ids]], uid)

    # Fetch payment methods
    payment_method_ids = list({payment.get('payment_method_id', [])[0] for payment in payments if payment.get('payment_method_id')})
    payment_methods = fetch_data_in_bulk('pos.payment.method', ['id', 'name'], [['id', 'in', payment_method_ids]], uid)
    payment_method_map = {method['id']: method.get('name') for method in payment_methods}

    # Insert data into SQL Server
    try:
        conn = pymssql.connect(
            server=server_name,
            user=sql_user_name,
            password=sql_password,
            database=database_name,
            charset='utf8'
        )
        cursor = conn.cursor()

        # Create table if it doesn't exist
        cursor.execute(f"""
        IF NOT EXISTS (
            SELECT * FROM INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_NAME = 'payments'
        )
        BEGIN
            CREATE TABLE payments (
                pay_id INT PRIMARY KEY,
                order_ref NVARCHAR(255),
                status NVARCHAR(255),
                payment_display_name NVARCHAR(255),
                payment_method NVARCHAR(255)
            );
        END
        """)
        conn.commit()

        print("Inserting data into SQL Server...")
        batch_data = []

        for order in orders:
            order_ref = order.get('name')
            status = order.get('state')
            payment_ids = order.get('payment_ids', [])

            for pay_id in payment_ids:
                payment = next((p for p in payments if p['id'] == pay_id), None)
                if not payment:
                    continue

                payment_display_name = payment.get('display_name')
                payment_method_id = payment.get('payment_method_id', [])[0] if payment.get('payment_method_id') else None
                payment_method = payment_method_map.get(payment_method_id, None)

                # Add to batch
                batch_data.append((pay_id, order_ref, status, payment_display_name, payment_method))

                # Insert in batches
                if len(batch_data) >= batch_size:
                    cursor.executemany(f"""
                    INSERT INTO payments (
                        pay_id, order_ref, status, payment_display_name, payment_method
                    ) VALUES (%s, %s, %s, %s, %s)
                    """, batch_data)
                    conn.commit()
                    batch_data = []

        # Final batch insert
        if batch_data:
            cursor.executemany(f"""
            INSERT INTO payments (
                pay_id, order_ref, status, payment_display_name, payment_method
            ) VALUES (%s, %s, %s, %s, %s)
            """, batch_data)
            conn.commit()

        print("All data inserted successfully.")

        # Close SQL connection
        cursor.close()
        conn.close()
    except Exception as e:
        print(f"Error during SQL operations: {e}")

# Run the function
process_and_insert()


Starting data synchronization...
Authenticated successfully. User ID: 112
Fetching POS orders...
Fetched 95742 orders.
Fetching related payments...
Inserting data into SQL Server...
All data inserted successfully.


### 4- Employee

In [35]:
import xmlrpc.client
import pymssql

# Custom transport with a timeout
class TimeoutTransport(xmlrpc.client.Transport):
    def __init__(self, timeout=None):
        super().__init__()
        self.timeout = timeout

    def make_connection(self, host):
        connection = super().make_connection(host)
        connection.timeout = self.timeout
        return connection

# Odoo Connection Details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# SQL Server Connection Details
server_name = 'SARAH\\SQLEXPRESS'
database_name = 'Odoo_sql_database'
sql_user_name = 'SuperAdmin'
sql_password = 'SuperAdmin'

# Date range filter
date_domain = [
    ('date_order', '>=', '2024-01-01'),
    ('date_order', '<=', '2024-12-31')
]

# Initialize batch size for SQL insert
batch_size = 100

def fetch_data_in_bulk(model, fields, uid, domain=None):
    """Fetch data from Odoo in bulk for a specific model."""
    try:
        models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object', transport=TimeoutTransport(timeout=300))
        data = models.execute_kw(
            db_name,
            uid,
            password,
            model,
            'search_read',
            [domain] if domain else [[]],
            {'fields': fields}
        )
        return data
    except Exception as e:
        print(f"Error fetching data from Odoo for model {model}: {e}")
        return []

def process_and_insert():
    print("Starting employee data synchronization from POS orders...")

    # Step 1: Authenticate with Odoo
    try:
        common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common', transport=TimeoutTransport(timeout=300))
        uid = common.authenticate(db_name, username, password, {})
        if not uid:
            print("Failed to authenticate. Check your credentials.")
            return
        print(f"Authenticated successfully. User ID: {uid}")
    except Exception as e:
        print(f"Authentication error: {e}")
        return

    # Step 2: Fetch employee data (user_id from pos.order)
    print("Fetching employee data from POS orders...")
    employees = fetch_data_in_bulk('pos.order', ['id', 'user_id'], uid)

    if not employees:
        print("No employees found in POS orders. Exiting.")
        return
    print(f"Fetched {len(employees)} POS orders.")

    # Step 3: Prepare and insert data into SQL Server
    try:
        conn = pymssql.connect(
            server=server_name,
            user=sql_user_name,
            password=sql_password,
            database=database_name,
            charset='utf8'
        )
        cursor = conn.cursor()

        # Create table if it doesn't exist
        cursor.execute(f"""
        IF NOT EXISTS (
            SELECT * FROM INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_NAME = 'employee'
        )
        BEGIN
            CREATE TABLE employee (
                emp_id INT PRIMARY KEY,
                emp_name NVARCHAR(255)
            );
        END
        """)
        conn.commit()

        print("Inserting employee data into SQL Server...")
        batch_data = []

        for order in employees:
            emp_id = order.get('user_id')  # Get the user_id (employee_id in the POS order)
            if emp_id:
                # Fetch employee name by using the user_id (as employee is linked to the user_id)
                emp_name = order.get('user_id')[1]  # user_id[1] contains the employee name

                # Add to batch
                batch_data.append((emp_id[0], emp_name))  # user_id[0] contains the employee ID

        # Insert in batches using MERGE to handle duplicates
        for emp in batch_data:
            cursor.execute(f"""
            MERGE INTO employee AS target
            USING (SELECT %d AS emp_id, %s AS emp_name) AS source
            ON target.emp_id = source.emp_id
            WHEN MATCHED THEN
                UPDATE SET target.emp_name = source.emp_name
            WHEN NOT MATCHED THEN
                INSERT (emp_id, emp_name)
                VALUES (source.emp_id, source.emp_name);
            """, (emp[0], emp[1]))
        conn.commit()

        print("All employee data from POS orders inserted successfully.")

        # Close SQL connection
        cursor.close()
        conn.close()
    except Exception as e:
        print(f"Error during SQL operations: {e}")

# Run the function
process_and_insert()


Starting employee data synchronization from POS orders...
Authenticated successfully. User ID: 112
Fetching employee data from POS orders...
Fetched 275420 POS orders.
Inserting employee data into SQL Server...
All employee data from POS orders inserted successfully.


### 5- Sales_Team

In [13]:
import xmlrpc.client
import pymssql

# Custom transport with a timeout
class TimeoutTransport(xmlrpc.client.Transport):
    def __init__(self, timeout=None):
        super().__init__()
        self.timeout = timeout

    def make_connection(self, host):
        connection = super().make_connection(host)
        connection.timeout = self.timeout
        return connection

# Odoo Connection Details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# SQL Server Connection Details
server_name = 'SARAH\\SQLEXPRESS'
database_name = 'Odoo_sql_database'
sql_user_name = 'SuperAdmin'
sql_password = 'SuperAdmin'

# Initialize batch size for SQL insert
batch_size = 100

def fetch_data_in_bulk(model, fields, uid, domain=None):
    """Fetch data from Odoo in bulk for a specific model."""
    try:
        models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object', transport=TimeoutTransport(timeout=300))
        data = models.execute_kw(
            db_name,
            uid,
            password,
            model,
            'search_read',
            [domain] if domain else [[]],
            {'fields': fields}
        )
        return data
    except Exception as e:
        print(f"Error fetching data from Odoo for model {model}: {e}")
        return []

def process_and_insert():
    print("Starting sales team data synchronization...")

    # Step 1: Authenticate with Odoo
    try:
        common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common', transport=TimeoutTransport(timeout=300))
        uid = common.authenticate(db_name, username, password, {})
        if not uid:
            print("Failed to authenticate. Check your credentials.")
            return
        print(f"Authenticated successfully. User ID: {uid}")
    except Exception as e:
        print(f"Authentication error: {e}")
        return

    # Step 2: Fetch sales team data
    print("Fetching sales team data...")
    sales_teams = fetch_data_in_bulk('crm.team', ['id', 'name'], uid)

    if not sales_teams:
        print("No sales teams fetched. Exiting.")
        return
    print(f"Fetched {len(sales_teams)} sales teams.")

    # Step 3: Prepare and insert data into SQL Server
    try:
        conn = pymssql.connect(
            server=server_name,
            user=sql_user_name,
            password=sql_password,
            database=database_name,
            charset='utf8'
        )
        cursor = conn.cursor()

        # Create table if it doesn't exist
        cursor.execute(f"""
        IF NOT EXISTS (
            SELECT * FROM INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_NAME = 'sales_team'
        )
        BEGIN
            CREATE TABLE sales_team (
                crm_team_id INT PRIMARY KEY,
                crm_team_name NVARCHAR(255)
            );
        END
        """)
        conn.commit()

        print("Inserting sales team data into SQL Server...")
        batch_data = []

        for team in sales_teams:
            crm_team_id = team.get('id')
            crm_team_name = team.get('name')  # The name can be in Arabic or other languages

            # Add to batch
            batch_data.append((crm_team_id, crm_team_name))

            # Insert in batches
            if len(batch_data) >= batch_size:
                cursor.executemany(f"""
                INSERT INTO sales_team (crm_team_id, crm_team_name)
                VALUES (%d, %s)
                """, batch_data)
                conn.commit()
                batch_data = []

        # Final batch insert
        if batch_data:
            cursor.executemany(f"""
            INSERT INTO sales_team (crm_team_id, crm_team_name)
            VALUES (%d, %s)
            """, batch_data)
            conn.commit()

        print("All sales team data inserted successfully.")

        # Close SQL connection
        cursor.close()
        conn.close()
    except Exception as e:
        print(f"Error during SQL operations: {e}")

# Run the function
process_and_insert()


Starting sales team data synchronization...
Authenticated successfully. User ID: 112
Fetching sales team data...
Fetched 4 sales teams.
Inserting sales team data into SQL Server...
All sales team data inserted successfully.


### 6- Customer

In [24]:
import xmlrpc.client
import pymssql

# Custom transport with a timeout
class TimeoutTransport(xmlrpc.client.Transport):
    def __init__(self, timeout=None):
        super().__init__()
        self.timeout = timeout

    def make_connection(self, host):
        connection = super().make_connection(host)
        connection.timeout = self.timeout
        return connection

# Odoo Connection Details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# SQL Server Connection Details
server_name = 'SARAH\\SQLEXPRESS'
database_name = 'Odoo_sql_database'
sql_user_name = 'SuperAdmin'
sql_password = 'SuperAdmin'

# Initialize batch size for SQL insert
batch_size = 100

def fetch_data_in_bulk(model, fields, uid, domain=None):
    """Fetch data from Odoo in bulk for a specific model."""
    try:
        models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object', transport=TimeoutTransport(timeout=300))
        data = models.execute_kw(
            db_name,
            uid,
            password,
            model,
            'search_read',
            [domain] if domain else [[]],
            {'fields': fields}
        )
        return data
    except Exception as e:
        print(f"Error fetching data from Odoo for model {model}: {e}")
        return []

def process_and_insert():
    print("Starting customer data synchronization...")

    # Step 1: Authenticate with Odoo
    try:
        common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common', transport=TimeoutTransport(timeout=300))
        uid = common.authenticate(db_name, username, password, {})
        if not uid:
            print("Failed to authenticate. Check your credentials.")
            return
        print(f"Authenticated successfully. User ID: {uid}")
    except Exception as e:
        print(f"Authentication error: {e}")
        return

    # Step 2: Fetch customer data
    print("Fetching customer data...")
    customers = fetch_data_in_bulk('res.partner', ['id', 'name'], uid)

    if not customers:
        print("No customers fetched. Exiting.")
        return
    print(f"Fetched {len(customers)} customers.")

    # Step 3: Prepare and insert data into SQL Server
    try:
        conn = pymssql.connect(
            server=server_name,
            user=sql_user_name,
            password=sql_password,
            database=database_name,
            charset='utf8'
        )
        cursor = conn.cursor()

        # Create table if it doesn't exist
        cursor.execute(f"""
        IF NOT EXISTS (
            SELECT * FROM INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_NAME = 'customer'
        )
        BEGIN
            CREATE TABLE customer (
                cust_id INT PRIMARY KEY,
                customer_name NVARCHAR(255)
            );
        END
        """)
        conn.commit()

        print("Inserting customer data into SQL Server...")
        batch_data = []

        for customer in customers:
            cust_id = customer.get('id')  # This is the Odoo record ID, mapped to partner_id
            customer_name = customer.get('name')  # The name can be in Arabic or other languages

            # Add to batch
            batch_data.append((cust_id, customer_name))

            # Insert in batches
            if len(batch_data) >= batch_size:
                cursor.executemany(f"""
                INSERT INTO customer (cust_id, customer_name)
                VALUES (%d, %s)
                """, batch_data)
                conn.commit()
                batch_data = []

        # Final batch insert
        if batch_data:
            cursor.executemany(f"""
            INSERT INTO customer (cust_id, customer_name)
            VALUES (%d, %s)
            """, batch_data)
            conn.commit()

        print("All customer data inserted successfully.")

        # Close SQL connection
        cursor.close()
        conn.close()
    except Exception as e:
        print(f"Error during SQL operations: {e}")

# Run the function
process_and_insert()


Starting customer data synchronization...
Authenticated successfully. User ID: 112
Fetching customer data...
Fetched 841 customers.
Inserting customer data into SQL Server...
All customer data inserted successfully.


### 7- product

In [ ]:
import xmlrpc.client
import pymssql

# Custom transport with a timeout
class TimeoutTransport(xmlrpc.client.Transport):
    def __init__(self, timeout=None):
        super().__init__()
        self.timeout = timeout

    def make_connection(self, host):
        connection = super().make_connection(host)
        connection.timeout = self.timeout
        return connection

# Odoo Connection Details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# SQL Server Connection Details
server_name = 'SARAH\\SQLEXPRESS'
database_name = 'Odoo_sql_database'
sql_user_name = 'SuperAdmin'
sql_password = 'SuperAdmin'

# Initialize batch size for SQL insert
batch_size = 100

def fetch_data_in_bulk(model, fields, uid, domain=None):
    """Fetch data from Odoo in bulk for a specific model."""
    try:
        models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object', transport=TimeoutTransport(timeout=300))
        data = models.execute_kw(
            db_name,
            uid,
            password,
            model,
            'search_read',
            [domain] if domain else [[]],
            {'fields': fields}
        )
        return data
    except Exception as e:
        print(f"Error fetching data from Odoo for model {model}: {e}")
        return []

def get_category_hierarchy(category_id, category_map):
    """Recursively fetch the category hierarchy and return the full category path."""
    category_hierarchy = []
    current_id = category_id
    
    while current_id:
        # Ensure we're working with an integer ID
        if isinstance(current_id, (list, tuple)):
            current_id = current_id[0] if current_id else None
        
        if current_id in category_map:
            category = category_map[current_id]
            category_hierarchy.insert(0, category['name'])  # Add category to the front of the list
            current_id = category.get('parent_id')  # Move to the parent category
            if isinstance(current_id, (list, tuple)):
                current_id = current_id[0] if current_id else None
        else:
            break
            
    return " / ".join(category_hierarchy) if category_hierarchy else "Unknown"

def process_and_insert():
    print("Starting product data synchronization...")

    # Step 1: Authenticate with Odoo
    try:
        common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common', transport=TimeoutTransport(timeout=300))
        uid = common.authenticate(db_name, username, password, {})
        if not uid:
            print("Failed to authenticate. Check your credentials.")
            return
        print(f"Authenticated successfully. User ID: {uid}")
    except Exception as e:
        print(f"Authentication error: {e}")
        return

    # Step 2: Fetch product data
    print("Fetching product data...")
    products = fetch_data_in_bulk('product.product', ['id', 'display_name', 'standard_price', 'categ_id', 'list_price'], uid)

    if not products:
        print("No products fetched. Exiting.")
        return
    print(f"Fetched {len(products)} products.")

    # Step 3: Fetch category data (including parent categories)
    print("Fetching product categories...")
    categories = fetch_data_in_bulk('product.category', ['id', 'name', 'parent_id'], uid)
    category_map = {cat['id']: cat for cat in categories}

    # Step 4: Prepare and insert data into SQL Server
    try:
        conn = pymssql.connect(
            server=server_name,
            user=sql_user_name,
            password=sql_password,
            database=database_name,
            charset='utf8'
        )
        cursor = conn.cursor()

        # Create table if it doesn't exist
        cursor.execute(f"""
        IF NOT EXISTS (
            SELECT * FROM INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_NAME = 'product'
        )
        BEGIN
            CREATE TABLE product (
                id INT PRIMARY KEY,
                product_name NVARCHAR(255),
                product_cost DECIMAL(18, 2),
                product_category NVARCHAR(1024),
                sales_price DECIMAL(18, 2)
            );
        END
        """)
        conn.commit()

        print("Inserting data into SQL Server...")
        batch_data = []

        for product in products:
            product_id = product.get('id')
            product_name = product.get('display_name')
            product_cost = product.get('standard_price', 0.0)
            
            # Handle categ_id which might be a list or None
            categ_id = product.get('categ_id')
            sales_price = product.get('list_price', 0.0)

            # Get the full category path using the category hierarchy function
            product_category = get_category_hierarchy(categ_id, category_map)

            # Add to batch
            batch_data.append((product_id, product_name, product_cost, product_category, sales_price))

            # Insert in batches
            if len(batch_data) >= batch_size:
                cursor.executemany(f"""
                INSERT INTO product (id, product_name, product_cost, product_category, sales_price)
                VALUES (%d, %s, %d, %s, %d)
                """, batch_data)
                conn.commit()
                batch_data = []

        # Final batch insert
        if batch_data:
            cursor.executemany(f"""
            INSERT INTO product (id, product_name, product_cost, product_category, sales_price)
            VALUES (%d, %s, %d, %s, %d)
            """, batch_data)
            conn.commit()

        print("All product data inserted successfully.")

        # Close SQL connection
        cursor.close()
        conn.close()
    except Exception as e:
        print(f"Error during SQL operations: {e}")

# Run the function
process_and_insert()